### Import Libraries

In [11]:
from pyspark.sql.functions import col, from_json, upper

from lib.database import get_postgres_table
from lib.spark import get_spark_session
from lib.sql import add_col_prefix

### Configure Session

In [12]:
spark = get_spark_session()
sc = spark.sparkContext

### Get Tables

In [15]:
activate_transactions = get_postgres_table(database="activate_core_production",
                                           partition_col="transaction_time_at",
                                           port=9037,
                                           table="transactions")
activate_transactions = add_col_prefix(activate_transactions, "atx")

activate_transaction_line_items = get_postgres_table(database="activate_core_production",
                                                     partition_col="transaction_time_at",
                                                     port=9037,
                                                     table="transaction_line_items")
activate_transaction_line_items = add_col_prefix(activate_transaction_line_items, "atxli")

oc_transactions = get_postgres_table(database="open_commerce",
                                     partition_col="startedAt",
                                     port=9036,
                                     table="transaction")
oc_transactions = add_col_prefix(oc_transactions, "octx")

oc_transaction_line_items = get_postgres_table(database="open_commerce",
                                               partition_col="createdAt",
                                               port=9036,
                                               table="transaction_item")
oc_transaction_line_items = add_col_prefix(oc_transaction_line_items, "octxli")

### Explore Data

In [16]:
activate_transactions = activate_transactions.withColumnRenamed("atx_id", "transaction_id") \
                                             .withColumnRenamed("atx_membership_id", "membership_id")
activate_transaction_line_items = activate_transaction_line_items.withColumnRenamed("atxli_id", "transaction_id") \
                                                                 .withColumnRenamed("atxli_membership_id", "membership_id")
activate_transactions = activate_transaction_line_items.join(activate_transactions, ["transaction_id", "membership_id"])

In [17]:
oc_transactions = oc_transactions.withColumnRenamed("octx_id", "transactionId")
oc_transaction_line_items = oc_transaction_line_items.withColumnRenamed("octxli_transactionId", "transactionId")
oc_transactions = oc_transaction_line_items.join(oc_transactions, ["transactionId"])

In [19]:
display(oc_transactions.limit(10).to_pandas_on_spark())

,transactionId,octxli_createdAt,octxli_updatedAt,octxli_id,octxli_uuid,octxli_qty,octxli_discountDescription,octxli_itemDescription,octxli_amountDisplayprice,octxli_amountPrice,octxli_taxDisplayprice,octxli_taxPrice,octxli_discountAppliedDisplayprice,octxli_discountAppliedPrice,octxli_refundedAmountDisplayprice,octxli_refundedAmountPrice,octxli_fuelItemId,octxli_storeItemId,octxli_carWashItemId,octx_createdAt,octx_updatedAt,octx_uuid,octx_customerUuid,octx_umti,octx_stac,octx_paymentProviderTransactionId,octx_startedAt,octx_completedAt,octx_status,octx_brandSpecificLocationId,octx_fuelingPosition,octx_authCode,octx_hardwareVendor,octx_transactionType,octx_paymentLevel,octx_loyaltyLevel,octx_fuelPumpStatus,octx_initiationResource,octx_receiptLines,octx_rawReceipt,octx_fuelReceipt,octx_amountDisplayprice,octx_amountPrice,octx_taxDisplayprice,octx_taxPrice,octx_refundedAmountDisplayprice,octx_refundedAmountPrice,octx_currencyCode,octx_currencySymbol,octx_additionalReceiptData,octx_preAuthAmountDisplayprice,octx_preAuthAmountPrice,octx_locationUuid,octx_primaryStatus,octx_secondaryStatus,octx_paymentInstrumentUuid,octx_stateChanges,octx_customerPreferences,octx_carWashDetails,octx_isPaymentProcessedExternally,octx_loyaltyIdentifier,octx_kountSessionId,octx_kountResponse,octx_posTransactionNumber,octx_sourceId,octx_cardType,octx_paymentMethod
0,5326,2022-01-14 14:56:07.766865,2022-01-14 14:56:07.792283,3859,d55c2ab7-27c4-415d-9174-99c08b7f998e,16.674,None,UNLD1,None,48.00,None,0.0,None,0.0,None,0.0,3825,NaN,NaN,2022-01-14 14:53:16.266041,2022-01-14 14:56:08.577769,c582b1c5-1deb-42a9-8a31-9b1a10c6e74f,355dc3cc-de58-453b-a80b-c3feb639dc53,97f183a8-5a7e-4b62-aaf5-d354388b6c19,None,574ab46d182a44dd9786a14a0488afaa,2022-01-14 14:53:16.266041,2022-01-14 14:56:08.566,STARTED,CEFCO #0414,15,None,None,AT_PUMP,None,None,None,FUEL_PUMP,"["" WELCOME "", "" CEFCO #414 ...","{""lines"":["" WELCOME "","" CEFCO #...","{""tax"": """", ""priceG"": ""2.879"", ""gallons"": ""16....",48,48.00,0.00,0.0,None,0.0,usd,$,None,$100.00,100.0,e4437651-1f0e-4c79-a51f-087400aaba56,COMPLETED,CLOSED,de61f928-789c-48b2-ba85-47720c9044c5,"[{""updatedAt"": ""2022-01-14T14:53:16.284Z"", ""pr...","{""printReceipt"": true, ""promptForCarWash"": null}",None,False,2512295461,None,None,9151965,NaN,None,None
1,5721,2022-01-19 19:15:29.566138,2022-01-19 19:15:29.586046,4120,7ec24b1b-2835-4d47-9403-cc01c717e2c5,7.174,None,UNLD1,None,20.51,None,0.0,None,0.0,None,0.0,4084,NaN,NaN,2022-01-19 19:14:08.440499,2022-01-19 19:15:30.697944,005d3aa1-e0e6-4580-8836-bd60598731bc,ab8bdaeb-64b9-481b-b0cc-19fe39423228,9f59779a-b354-4acb-a213-2d12e77efa7f,None,2d70fe309b1643d19c4c7a72f814206f,2022-01-19 19:14:08.440499,2022-01-19 19:15:30.685,STARTED,CEFCO #0071,5,None,None,AT_PUMP,None,None,None,FUEL_PUMP,"["" WELCOME TO "", "" CEFCO #0071 ...","{""lines"":["" WELCOME TO "","" CEFCO #0...","{""tax"": """", ""priceG"": ""2.859"", ""gallons"": ""7.1...",20.51,20.51,0.00,0.0,None,0.0,usd,$,None,$100.00,100.0,fa5a50d5-bb1f-462d-b7ed-7d516c318197,COMPLETED,CLOSED,9746c89a-5436-4b19-afec-31eff0f8fa78,"[{""updatedAt"": ""2022-01-19T19:14:08.482Z"", ""pr...","{""printReceipt"": false, ""promptForCarWash"": null}",None,False,2544215186,None,None,9057079,NaN,None,None
2,6234,2022-01-25 14:04:18.473136,2022-01-25 14:04:18.493798,4448,05551a3a-614f-47ab-8ac3-09556837c129,7.154,None,UNLD1,None,20.60,None,0.0,None,0.0,None,0.0,4412,NaN,NaN,2022-01-25 14:02:35.622716,2022-01-25 14:04:19.463756,633badff-7035-4cf1-b25b-578f5da6ec50,8671a404-6ec3-4a07-937f-401add12bc0a,d2d1c7a6-cdee-44d6-9223-1eb23ae1f8c5,None,6524fe5ae118406ea8698e2164252c71,2022-01-25 14:02:35.622716,2022-01-25 14:04:19.446,STARTED,CEFCO #0414,16,None,None,AT_PUMP,None,None,None,FUEL_PUMP,"["" WELCOME "", "" CEFCO #414 ...","{""lines"":["" WELCOME "","" CEFCO #...","{""tax"": """", ""priceG"": ""2.879"", ""gallons"": ""7.1...",20.6,20.60,0.00,0.0,None,0.0,usd,$,None,$100.00,100.0,e4437651-1f0e-4c79-a51f-087400aaba56,COMPLETED,CLOSED,038bde

In [ ]:
display(activate_transactions.limit(10).to_pandas_on_spark())